In [1]:
from bs4 import BeautifulSoup
import bs4
import requests
import pandas as pd
import re
from nltk.tokenize import sent_tokenize
from nltk.tokenize import word_tokenize
from nltk.probability import FreqDist
import matplotlib.pyplot as plt
from nltk.corpus import stopwords
import string

In [9]:
def remove_html_tags(text):
    clean1 = re.compile('<a.+?/a>')
    clean2 = re.compile('<.+?>')
    text = re.sub(clean2, '', re.sub(clean1, '', str(text)))
    text = ''.join([i if i in printable else i.replace(i,"_") for i in text])
    text = re.sub(r'\.(?! )', '. ', re.sub(r' +', ' ', re.sub('_+', " ", text)))
    return text

def clean_roles(text):
    clean = re.compile('<p.+?/p>')
    clean2 = re.compile('<.+?>')
    return re.sub(clean2, '', re.sub(clean, "", str(text), 1))
    
printable = set(string.printable)

url = "https://www.churchofjesuschrist.org/general-conference/conferences?lang=eng"
response = requests.get(url)
soup = BeautifulSoup(response.text, 'lxml')
years = soup.find_all("a", {'class':"year-line__link"})
years_links = []
for year in years:
    years_links.append(year.get('href'))
    
    
final_talks = []
final_titles = []
final_speakers = []
final_years = []
final_months = []
final_position = []
start = "/general-conference/"
end  = "?lang=eng"

### Main

for i, year in enumerate(years_links):
    if i > 9:
        pass
    try:   
        result = re.search(start + '(.*)' + end, year)
        result = re.sub("\?", "", re.sub("/" ,", ",result.group(1))).partition(',')
    except:
        print("Done or Broke")
        break
        
    print('Starting {} {}, {}'.format(i, result[0],  result[2]), end=' ')
    
    url = "https://www.churchofjesuschrist.org" + year
    response = requests.get(url)
    soup = BeautifulSoup(response.text, 'lxml')
    
    titles = soup.find_all("div", class_="lumen-tile__title")
    for title in titles:
        final_titles.append(title.get_text().strip('\n'))
        final_months += [result[2]]
        final_years += [result[0]]
    
    speakers = soup.find_all("div", class_="lumen-tile__content")
    for speaker in speakers:
        final_speakers.append(speaker.text)
        
    links = soup.find_all("a", class_="lumen-tile__link")
    link_list = []
    for link in links:
        link_list.append(link.get('href'))
    talks_list = []
    position_list = []
    for link in range(len(link_list)):
        print(" --- {} of {} Talks".format(link+1, len(link_list)), sep=' ', end='\r', flush=True)
        url = "https://www.churchofjesuschrist.org"
        url += link_list[link]
        page_response = requests.get(url)
        page_soup = BeautifulSoup(page_response.text)
        position = page_soup.find("p",{'class':"author-role"})
        if position == None:
            position = page_soup.find("div",{'class':"byline"})
            position = clean_roles(position)
        else:
            position = position.text
        final_position.append(position)
        talks = page_soup.find_all("div", {'class':"body-block"})
        for talk in talks:
            text = remove_html_tags(talk)
        final_talks.append(text)
            
    while len(final_talks) < len(final_speakers):
        final_talks.append("NA")
    print()

Starting 0 2019,  04    --- 33 Talks
33
Starting 1 2019,  10    --- 35 Talks
35
Starting 2 2018,  04    --- 38 Talks
38
Starting 3 2018,  10    --- 34 Talks
34
Starting 4 2017,  04    --- 39 Talks
39
Starting 5 2017,  10    --- 36 Talks
36
Starting 6 2016,  04    --- 40 Talks
40
Starting 7 2016,  10    --- 38 Talks
38
Starting 8 2015,  04    --- 40 Talks
40
Starting 9 2015,  10    --- 42 Talks
42
Starting 10 2014,  04    --- 42 Talks
42
Starting 11 2014,  10    --- 40 Talks
40
Starting 12 2013,  04    --- 41 Talks
41
Starting 13 2013,  10    --- 39 Talks
39
Starting 14 2012,  04    --- 41 Talks
41
Starting 15 2012,  10    --- 39 Talks
39
Starting 16 2011,  04    --- 41 Talks
41
Starting 17 2011,  10    --- 39 Talks
39
Starting 18 2010,  04    --- 41 Talks
41
Starting 19 2010,  10    --- 39 Talks
39
Starting 20 2009,  04    --- 40 Talks
40
Starting 21 2009,  10    --- 39 Talks
39
Starting 22 2008,  04    --- 43 Talks
43
Starting 23 2008,  10    --- 37 Talks
37
Starting 24 2007,  04    -

In [10]:
len(final_years), len(final_months), len(final_speakers), len(final_titles), len(final_talks), len(final_position)

(3839, 3839, 3839, 3839, 3839, 3839)

In [11]:
data_dict = {"Year":final_years, "Month":final_months, "Speaker":final_speakers, "Role":final_position, "Title":final_titles, "Talk":final_talks}
data = pd.DataFrame(data_dict)
data.head()

,Year,Month,Speaker,Role,Title,Talk
0,2019,04,Ulisses Soares,Of the Quorum of the Twelve Apostles,How Can I Understand?,"My dear brothers and sisters, what a great joy..."
1,2019,04,Becky Craven,Second Counselor in the Young Women General Pr...,Careful versus Casual,"I once saw a sign in a store window that said,..."
2,2019,04,Brook P. Hales,Of the Seventy,Answers to Prayer,An important and comforting doctrine of the go...
3,2019,04,Dieter F. Uchtdorf,Of the Quorum of the Twelve Apostles,Missionary Work: Sharing What Is in Your Heart,Last month the Twelve were invited by our dear...
4,2019,04,W. Christopher Waddell,Second Counselor in the Presiding Bishopric,Just as He Did,"Approximately 18 months ago, in the fall of 20..."


In [12]:
data.to_csv('GenConfTalks.csv', index=False) 